
# Classification

The following example shows how to fit a simple classification model with
*auto-sklearn*.


In [1]:
from pprint import pprint
import sklearn.metrics
import autosklearn.classification
import pandas as pd
from sklearn.model_selection import train_test_split
import autosklearn

## Data Loading



In [2]:

# file_path = '../data/titanic_dirty_data.csv'
file_path = '../data/airbnb.csv'


df = pd.read_csv(file_path)

# df = df.dropna(subset=['Survived'])
# y = df['Survived']
# X = df.drop('Survived', axis=1)

df = df.dropna(subset=['Rating'])
y = df['Rating']
X = df.drop('Rating', axis=1)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Build and fit a classifier



In [3]:
automl = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=120,
    per_run_time_limit=10,
    include = {
        'classifier': ["mlp"],
    },
    tmp_folder="tmp/autosklearn_classification_example_tmp5",
)

automl.fit(X_train, y_train)
run_key = list(automl.automl_.runhistory_.data.keys())[0]
run_value = automl.automl_.runhistory_.data[run_key]
config=automl.automl_.runhistory_.ids_config[run_key.config_id]
print(config)

Fitting to the training data:   0%|          | 0/120 [00:00<?, ?it/s, The total time budget for this task is 0:02:00]

/home/preethi/projects/hitlda-project/auto-sklearn/autosklearn/data/feature_validator.py:298: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(X[column]):
/home/preethi/projects/hitlda-project/auto-sklearn/autosklearn/data/feature_validator.py:318: UserWarning: Input Column LocationName has generic type object. Autosklearn will treat this column as string. Please ensure that this setting is suitable for your task.
  warnings.warn(


MLPP INITTT


Fitting to the training data: 100%|██████████| 120/120 [01:50<00:00,  1.09it/s, The total time budget for this task is 0:02:00]


Configuration(values={
  'balancing:strategy': 'none',
  'classifier:__choice__': 'mlp',
  'classifier:mlp:activation': 'relu',
  'classifier:mlp:alpha': 0.0001,
  'classifier:mlp:batch_size': 'auto',
  'classifier:mlp:beta_1': 0.9,
  'classifier:mlp:beta_2': 0.999,
  'classifier:mlp:early_stopping': 'valid',
  'classifier:mlp:epsilon': 1e-08,
  'classifier:mlp:hidden_layer_depth': 5,
  'classifier:mlp:learning_rate_init': 0.001,
  'classifier:mlp:n_iter_no_change': 32,
  'classifier:mlp:num_nodes_per_layer': 256,
  'classifier:mlp:shuffle': 'True',
  'classifier:mlp:solver': 'adam',
  'classifier:mlp:tol': 0.0001,
  'classifier:mlp:validation_fraction': 0.1,
  'data_preprocessor:__choice__': 'feature_type',
  'data_preprocessor:feature_type:numerical_transformer:imputation:strategy': 'mean',
  'data_preprocessor:feature_type:numerical_transformer:rescaling:__choice__': 'standardize',
  'data_preprocessor:feature_type:text_transformer:text_encoding:__choice__': 'tfidf_encoding',
  'dat

## View the models found by auto-sklearn



In [4]:
print(automl.leaderboard())

          rank  ensemble_weight type      cost  duration
model_id                                                
10           1             0.54  mlp  0.306072  6.106134
2            2             0.32  mlp  0.320274  6.938709
6            3             0.14  mlp  0.320764  7.437046


## Print the final ensemble constructed by auto-sklearn



In [5]:
pprint(automl.show_models(), indent=4)

{   2: {   'balancing': Balancing(random_state=1),
           'classifier': <autosklearn.pipeline.components.classification.ClassifierChoice object at 0x7f112c4ab760>,
           'cost': 0.3202742409402547,
           'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7f112c4aafa0>,
           'ensemble_weight': 0.32,
           'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x7f112c4ae610>,
           'model_id': 2,
           'rank': 2,
           'sklearn_classifier': MLPClassifier(beta_1=0.999, beta_2=0.9, early_stopping=True,
              hidden_layer_sizes=(64, 64, 64), max_iter=32, n_iter_no_change=32,
              random_state=1, verbose=0, warm_start=True)},
    6: {   'balancing': Balancing(random_state=1, strategy='weighting'),
           'classifier': <autosklearn.pipeline.components.classification.ClassifierChoice object at 0x7f112c4bb670>,
       

## Get the Score of the final ensemble



In [6]:
predictions = automl.predict(X_test)
print(set(predictions))
print(set(y_test))
print("Accuracy score:", sklearn.metrics.accuracy_score(y_test, predictions))
print("Precision", sklearn.metrics.precision_score(y_test, predictions,pos_label='Y'))
print("Recall", sklearn.metrics.recall_score(y_test, predictions,pos_label='Y'))


/home/preethi/projects/hitlda-project/auto-sklearn/py39/lib/python3.9/site-packages/sklearn/utils/validation.py:571: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  array.dtypes.apply(is_sparse).any()):
/home/preethi/projects/hitlda-project/auto-sklearn/py39/lib/python3.9/site-packages/sklearn/utils/validation.py:571: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  array.dtypes.apply(is_sparse).any()):
/home/preethi/projects/hitlda-project/auto-sklearn/py39/lib/python3.9/site-packages/sklearn/utils/validation.py:571: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  array.dtypes.apply(is_sparse).any()):


{'N', 'Y'}
{'N', 'Y'}
Accuracy score: 0.6815341521223874
Precision 0.703558282208589
Recall 0.9139305068536818


In [7]:

from sklearn.ensemble import RandomForestClassifier
import numpy as np
import matplotlib.pyplot as plt

# file_path = '../data/clean_titanic_data_rf.csv'
file_path = '../data/clean_airbnb_data_mlp.csv'


df = pd.read_csv(file_path)

# df = df.dropna(subset=['Survived'])
# y = df['Survived']
# X = df.drop('Survived', axis=1)

df = df.dropna(subset=['Rating'])
y = df['Rating']
X = df.drop('Rating', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

feature_names = [f"feature {i}" for i in range(X.shape[1])]
forest = RandomForestClassifier(random_state=0)
forest.fit(X_train, y_train)

importances = forest.feature_importances_
print(len(importances))
sorted_indices = np.argsort(importances)[::-1]
print(sorted_indices)


/home/preethi/projects/hitlda-project/auto-sklearn/py39/lib/python3.9/site-packages/sklearn/utils/validation.py:571: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  array.dtypes.apply(is_sparse).any()):


138
[  6   7   5   4   3   2   1   0  33   8  35  29  12  13  34  15  31  30
  37  22  32  20  27  10  16  23  25  24  28   9  19  36  26  11  17  21
  18  14  72  84 102 128  74 131 137 132 136  87 133 122 118 116 135 126
  43  64  94  73  90  78 127 130 129 121 134 110 109  97  54 117  75  86
 111  68 107  59  58  85  38 112  93 113  55  57 125 106  98  77  99 108
 120 119 100  92  40  91 124 123  82  47 115  88  81 114  51  48  45  39
  61 105  53  62 101  66 104  96 103  49  95  44  41  69  52  46  80  56
  60  70  79  42  63  89  50  76  65  83  71  67]


In [8]:
rf_predictions = forest.predict(X_test)
print(set(rf_predictions))
print(set(y_test))
print("Accuracy score:", sklearn.metrics.accuracy_score(y_test, rf_predictions))
print("Precision", sklearn.metrics.precision_score(y_test, rf_predictions))
print("Recall", sklearn.metrics.recall_score(y_test, rf_predictions))

{0.0, 1.0}
{0.0, 1.0}
Accuracy score: 0.7274294333117862
Precision 0.7622882532629826
Recall 0.8703233988585922


/home/preethi/projects/hitlda-project/auto-sklearn/py39/lib/python3.9/site-packages/sklearn/utils/validation.py:571: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  array.dtypes.apply(is_sparse).any()):
